<a href="https://colab.research.google.com/github/EnzoZhu315/EAN-13-Barcode-Automation-System/blob/main/YuHongZhu_EAN13_Barcodes_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Enzo-Barcodes Generation V3.0**

**使用教程：在Google Drive上先填写信息，然后回到此界面，点击运行按钮，等待文件的爬取，之后点击下载PDF按钮，即可生成条形码的PDF格式到网页下载中**

In [ ]:
# @title 🏷️ EAN-13 自动化生成器 (发布版) { display-mode: "form" }

# @markdown ---
# @markdown ### 📥 第一步：输入 P 号码
P_Input = "P18128435" # @param {type:"string"}

# @markdown ---
# @markdown ### 📂 第二步：云端配置
# @markdown 请输入 Google Sheets 的完整名称
Sheet_Name = "条形码信息输入表" # @param {type:"string"}
# @markdown 请输入 PDF 模板所在的文件夹路径 (例如: /content/drive/Shareddrives/your_folder/)
Folder_Path = "/content/drive/Shareddrives/barcode generation/" # @param {type:"string"}

import os
from google.colab import drive, auth, files
import gspread
from google.auth import default
from IPython.display import display, clear_output
import ipywidgets as widgets

# 环境静默安装
try:
    from reportlab.lib.units import mm
    from pypdf import PdfReader, PdfWriter
except ImportError:
    os.system("pip install reportlab pypdf gspread --upgrade -q")
    os.system("apt-get install -y fonts-wqy-microhei -q")

from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.graphics.barcode import eanbc
from reportlab.graphics.shapes import Drawing
from reportlab.graphics import renderPDF
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from pypdf import PdfReader, PdfWriter

# 1. 挂载 Google Drive (标准路径)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. 授权 Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 注册中文字体
FONT_PATH = "/usr/share/fonts/truetype/wqy/wqy-microhei.ttc"
if os.path.exists(FONT_PATH):
    pdfmetrics.registerFont(TTFont('SimHei', FONT_PATH))

def generate_process(b):
    clear_output(wait=True)
    print(f"🌐 正在从云端检索 P号码: {P_Input} ...")

    try:
        # 打开表格并读取数据
        sh = gc.open(Sheet_Name)
        worksheet = sh.get_worksheet(0)
        all_rows = worksheet.get_all_values()

        if not all_rows:
            print("❌ 错误：表格内容为空！")
            return

        header = [str(i).strip() for i in all_rows[0]]

        # 动态定位列索引 (不因列顺序改变而失效)
        try:
            idx_p = next(i for i, v in enumerate(header) if "P号码" in v)
            idx_sku = next(i for i, v in enumerate(header) if "SKU号码" in v)
            idx_desc = next(i for i, v in enumerate(header) if "描述" in v)
            idx_code = next(i for i, v in enumerate(header) if "条码号码" in v)
        except StopIteration:
            print(f"❌ 错误：表头格式不符。识别到：{header}")
            return

        # 匹配 P 号码记录
        target_row = None
        for row in all_rows[1:]:
            if len(row) > idx_p and str(row[idx_p]).strip() == str(P_Input).strip():
                target_row = row
                break

        if not target_row:
            print(f"❌ 错误：找不到 P号码 [{P_Input}]。")
            return

        # 提取字段
        P_Num = str(target_row[idx_p]).strip()
        SKU_Num = str(target_row[idx_sku]).strip()
        Desc_Val = str(target_row[idx_desc]).strip()
        Code_Raw = str(target_row[idx_code]).strip()
        Code_Val = ''.join(filter(str.isdigit, Code_Raw))

        print(f"✅ 匹配成功：P {P_Num} | SKU {SKU_Num}")

        # --- 寻找 PDF 模板 ---
        pdf_path = None
        if os.path.exists(Folder_Path):
            for f in os.listdir(Folder_Path):
                if "标签套准线标记" in f and f.endswith(".pdf"):
                    pdf_path = os.path.join(Folder_Path, f)
                    break

        if not pdf_path:
            print(f"❌ 错误：在路径下找不到 PDF 模板文件。")
            return

        # --- 渲染逻辑 ---
        reader = PdfReader(pdf_path)
        page = reader.pages[0]
        sw, sh_pdf = float(page.mediabox.width), float(page.mediabox.height)
        OVERLAY, OUTPUT = "temp_ov.pdf", f"Label_{P_Num}.pdf"
        c = canvas.Canvas(OVERLAY, pagesize=(sw, sh_pdf))

        LABEL_W, LABEL_H = 70 * mm, 50 * mm
        GAP_X, GAP_Y = 3.025 * mm, 6 * mm
        BASE_X = (sw - 292.1 * mm) / 2
        BASE_Y_TOP = (sh_pdf + 106 * mm) / 2

        last_box_rx, top_y_a = 0, 0

        for row in range(2):
            for col in range(4):
                x = BASE_X + 1.5125 * mm + col * (LABEL_W + GAP_X)
                y_t = BASE_Y_TOP - 0.1 * mm - row * (LABEL_H + GAP_Y)
                y_b = y_t - LABEL_H
                if row == 0:
                    top_y_a = y_t
                    if col == 3: last_box_rx = x + LABEL_W

                # 绘图逻辑 (继承 V8.0 间距标准)
                c.setStrokeColorRGB(1, 0, 0); c.setLineWidth(0.15 * mm); c.rect(x, y_b, LABEL_W, LABEL_H)
                c.setFillColorRGB(0, 0, 0); c.setStrokeColorRGB(0, 0, 0)

                ty = y_t - 2.7 * mm - 8
                c.saveState(); c.setLineWidth(0.3)
                c.setFont('SimHei', 9)
                c.drawString(x+2.2*mm, ty, P_Num)
                sw_sku = c.stringWidth(SKU_Num, 'SimHei', 9)
                c.drawString(x+LABEL_W-2.2*mm-sw_sku, ty, SKU_Num)
                c.restoreState()

                dy = y_t - 9.7 * mm
                c.saveState(); c.setLineWidth(0.15); dw = c.stringWidth(Desc_Val, 'SimHei', 5.5)
                c.setFont('SimHei', 5.5)
                c.drawString(x+(LABEL_W-dw)/2, dy, Desc_Val)
                c.restoreState()

                bch_s, g_e = 31.915 * mm, 2.0 * mm; bch_f = bch_s + g_e
                bcw = 45.474 * mm; mod = bcw / 95.0
                bcx = x + (LABEL_W-bcw)/2 - (mod*11); bcy = dy - 4*mm - bch_f + 5
                barcode = eanbc.Ean13BarcodeWidget(Code_Val)
                barcode.barHeight, barcode.barWidth, barcode.humanReadable = bch_f, mod, False
                d = Drawing(bcw, bch_f); d.add(barcode); renderPDF.draw(d, c, bcx, bcy)

                c.saveState(); c.setFillColorRGB(1, 1, 1); bx_r = bcx + (mod*11)
                c.rect(bx_r + (3*mod), bcy, 42*mod, g_e, stroke=0, fill=1)
                c.rect(bx_r + (50*mod), bcy, 42*mod, g_e, stroke=0, fill=1)
                c.restoreState()

                ny = bcy - 1.5 * mm
                c.saveState(); c.setFont('SimHei', 9.5)
                c.drawString(bx_r-4.2*mm, ny, Code_Val[0])
                for i in range(6): c.drawString(bx_r+(3+i*7+0.8)*mod, ny, Code_Val[1+i])
                for i in range(6): c.drawString(bx_r+(50+i*7+0.8)*mod, ny, Code_Val[7+i])
                c.drawString(bx_r+bcw+1.2*mm, ny, ">")
                c.restoreState()

                if row == 0:
                    c.setFillColorRGB(0,0,0); c.rect(x+62*mm, y_b-4.5*mm, 5*mm, 3*mm, fill=1)

        h_s = f"{P_Num} {Desc_Val} 92 T"
        c.saveState(); c.setFont('SimHei', 5.5); c.translate(last_box_rx, top_y_a+1.5*mm); c.rotate(180)
        c.drawString(5*mm, -2.0*mm, h_s); c.restoreState()
        c.save()

        overlay_r = PdfReader(OVERLAY); page.merge_page(overlay_r.pages[0])
        writer = PdfWriter(); writer.add_page(page)
        with open(OUTPUT, "wb") as f_out: writer.write(f_out)

        clear_output()
        print(f"✅ 生成成功！文件名为: {OUTPUT}")
        dl_btn = widgets.Button(description="📂 下载 PDF", button_style='success')
        dl_btn.on_click(lambda b: files.download(OUTPUT)); display(dl_btn)

    except Exception as e:
        print(f"❌ 运行失败: {e}")

main_btn = widgets.Button(description="🚀 一键生成", button_style='primary', layout=widgets.Layout(width='300px', height='50px'))
main_btn.on_click(generate_process); display(main_btn)

✅ 生成成功！
已直接联动 Google Sheet: 条形码信息输入表


Button(button_style='success', description='📂 下载 PDF', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>